In [1]:
import pandas as pd

df = pd.read_csv('dataset/synthetic_logs.csv')
df

,timestamp,source,log_message,target_label,complexity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert
...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert


In [2]:
df.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM'], dtype=object)

In [3]:
df.target_label.unique()

array(['HTTP Status', 'Critical Error', 'Security Alert', 'Error',
       'System Notification', 'Resource Usage', 'User Action',
       'Workflow Error', 'Deprecation Warning'], dtype=object)

Let's implement log message clustering using DBSCAN algorithm with sentence transformers for text embedding. We'll:
1. Install required packages
2. Create embeddings using sentence-transformers
3. Apply DBSCAN clustering
4. Analyze the results

In [4]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
import numpy as np

# Load the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings
embeddings = model.encode(df['log_message'].tolist(), show_progress_bar=True)

C:\Users\othma\PyCharmMiscProject\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 76/76 [00:59<00:00,  1.28it/s]


In [6]:
# Apply DBSCAN clustering
clustering = DBSCAN(eps=0.5, min_samples=5).fit(embeddings)

# Add cluster labels to dataframe
df['cluster'] = clustering.labels_
df

,timestamp,source,log_message,target_label,complexity,cluster
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,-1
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,-1
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0
...,...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert,0
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert,-1
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert,9
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert,-1


In [7]:
# Show clusters distribution
print(f"Number of unique clusters: {len(np.unique(clustering.labels_))}")
print("\nCluster distribution:")
print(df['cluster'].value_counts().sort_index())

Number of unique clusters: 29

Cluster distribution:
cluster
-1     548
 0     809
 1      42
 2      53
 3      84
 4      60
 5      31
 6      15
 7      99
 8      86
 9     206
 10     10
 11     48
 12     42
 13     58
 14     14
 15     29
 16     51
 17     21
 18     15
 19      7
 20     20
 21     17
 22      7
 23      8
 24      6
 25     11
 26      6
 27      7
Name: count, dtype: int64


In [8]:
# Display sample messages from each cluster
for cluster in sorted(df['cluster'].unique()):
    print(f"\nCluster {cluster}:")
    sample_messages = df[df['cluster'] == cluster]['log_message'].sample(
        min(3, len(df[df['cluster'] == cluster]))).values
    for msg in sample_messages:
        print(f"- {msg}")


Cluster -1:
- IP 192.168.176.1 blocked due to potential attack
- User 7662 tried to bypass API security measures
- Security warning: unusual network activity on server 41

Cluster 0:
- nova.osapi_compute.wsgi.server [req-2d362d97-cb67-4511-9150-17dccfff1ffc 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" Return code: 200 len: 1893 time: 0.2728651
- nova.osapi_compute.wsgi.server [req-4d05bae9-8a34-42b6-98be-991eae0f992c 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" Return code: 200 len: 1874 time: 0.1925120
- nova.osapi_compute.wsgi.server [req-1a40db47-25b1-456f-82a6-62af7d642a7b 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" Status code -  200 len: 1893 time: 0.2719719

Cluste

Note: 
- Cluster -1 represents noise points (outliers)
- You may need to adjust eps and min_samples parameters of DBSCAN based on your specific needs
- The chosen model 'all-MiniLM-L6-v2' is a good balance between speed and performance


Let's create a regex-based log classification system based on the target labels from the synthetic dataset.
First, we'll analyze the labels and create patterns for each category.

In [9]:
import re

# Define regex patterns for each log category
log_patterns = {
    r"User User\d+ logged (in|out).": "User Action",
    r"Backup (started|ended) at .*": "System Notification",
    r"Backup completed successfully.": "System Notification",
    r"System updated to version .*": "System Notification",
    r"File .* uploaded successfully by user .*": "System Notification",
    r"Disk cleanup completed successfully.": "System Notification",
    r"System reboot initiated by user .*": "System Notification",
    r"Account with ID .* created by .*": "User Action"
}

def classify_log(message):
    for pattern, category in log_patterns.items():
        if re.search(pattern, message):
            return category
    return None

In [10]:
# Apply classification to the dataset
df['regex_label'] = df['log_message'].apply(classify_log)

# Calculate accuracy
accuracy = (df['regex_label'] == df['target_label']).mean()
print(f"Classification Accuracy: {accuracy:.2%}")

# Show confusion matrix
confusion = pd.crosstab(df['target_label'], df['regex_label'],
                        margins=True)
confusion

Classification Accuracy: 20.75%


regex_label,System Notification,User Action,All
target_label,,,
System Notification,356,0,356
User Action,0,144,144
All,356,144,500


In [11]:
classify_log("User User123 logged in.")

'User Action'

In [12]:
df[df['regex_label'].notnull()]

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by us...,System Notification,regex,2,System Notification
14,1/4/2025 1:43,ThirdPartyAPI,File data_3847.csv uploaded successfully by us...,System Notification,regex,2,System Notification
15,5/1/2025 9:41,ModernCRM,Backup completed successfully.,System Notification,regex,4,System Notification
18,2/22/2025 17:49,ModernCRM,Account with ID 5351 created by User634.,User Action,regex,5,User Action
27,9/24/2025 19:57,ThirdPartyAPI,User User685 logged out.,User Action,regex,7,User Action
...,...,...,...,...,...,...,...
2376,6/27/2025 8:47,ModernCRM,System updated to version 2.0.5.,System Notification,regex,13,System Notification
2381,9/5/2025 6:39,ThirdPartyAPI,Disk cleanup completed successfully.,System Notification,regex,16,System Notification
2394,4/3/2025 13:13,ModernHR,Disk cleanup completed successfully.,System Notification,regex,16,System Notification
2395,5/2/2025 14:29,ThirdPartyAPI,Backup ended at 2025-05-06 11:23:16.,System Notification,regex,8,System Notification


In [13]:
df[df['regex_label'].isnull()]

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,-1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,-1,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None
...,...,...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert,0,None
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert,-1,None
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert,9,None
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert,-1,None


we can see that we matched just rows with regex complexity

Stage 2: Classification Using Embeddings

In [14]:
df_non_regex = df[df['regex_label'].isnull()].copy()
df_non_regex.shape

(1910, 7)

In [15]:
df_legacy = df_non_regex[df_non_regex.source=="LegacyCRM"]
df_legacy

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
60,2025-10-06 16:55:23,LegacyCRM,Lead conversion failed for prospect ID 7842 du...,Workflow Error,llm,-1,None
255,2025-05-03 16:55:35,LegacyCRM,API endpoint 'getCustomerDetails' is deprecate...,Deprecation Warning,llm,-1,None
377,2025-06-24 12:16:29,LegacyCRM,Customer follow-up process for lead ID 5621 fa...,Workflow Error,llm,-1,None
1325,2025-04-17 07:33:44,LegacyCRM,Escalation rule execution failed for ticket ID...,Workflow Error,llm,-1,None
1734,2025-04-30 07:47:30,LegacyCRM,The 'ExportToCSV' feature is outdated. Please ...,Deprecation Warning,llm,-1,None
1826,2025-01-23 10:33:36,LegacyCRM,Support for legacy authentication methods will...,Deprecation Warning,llm,-1,None
2217,2025-05-12 09:46:54,LegacyCRM,Task assignment for TeamID 3425 could not comp...,Workflow Error,llm,-1,None


In [16]:
df_non_legacy = df_non_regex[df_non_regex.source!="LegacyCRM"]
df_non_legacy

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,-1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,-1,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None
...,...,...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert,0,None
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert,-1,None
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert,9,None
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert,-1,None


In [17]:
df_non_legacy.shape

(1903, 7)

In [18]:
# generate embeddings
filtered_embeddings = model.encode(df_non_legacy['log_message'].tolist(), show_progress_bar=True)

Batches: 100%|██████████| 60/60 [01:02<00:00,  1.04s/it]


In [19]:
filtered_embeddings[:2]

array([[-1.02939717e-01,  3.35459895e-02, -2.20260434e-02,
         1.55100622e-03, -9.86912940e-03, -1.78956345e-01,
        -6.34409934e-02, -6.01761267e-02,  2.81108730e-02,
         5.99619709e-02, -1.72618106e-02,  1.43368833e-03,
        -1.49559975e-01,  3.15280259e-03, -5.66031225e-02,
         2.71685645e-02, -1.49890687e-02, -3.54037769e-02,
        -3.62936147e-02, -1.45410709e-02, -5.61491819e-03,
         8.75539333e-02,  4.55120951e-02,  2.50963438e-02,
         1.00188032e-02,  1.24267070e-02, -1.39923558e-01,
         7.68695921e-02,  3.14095505e-02, -4.15250845e-03,
         4.36903723e-02,  1.71250105e-02, -8.00951421e-02,
         5.74006326e-02,  1.89091861e-02,  8.55261832e-02,
         3.96398939e-02, -1.34371832e-01, -1.44363695e-03,
         3.06711602e-03,  1.76854104e-01,  4.44891676e-03,
        -1.69273838e-02,  2.24266760e-02, -4.35050540e-02,
         6.09031972e-03, -9.98173840e-03, -6.23971745e-02,
         1.07371574e-02, -6.04891405e-03, -7.14659989e-0

To train a logistic regression model using as features (`X`) and `target_label` as the target variable (`y`), `filtered_embeddings`

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Define X (features) and y (target)
X = filtered_embeddings
y = df_non_legacy['target_label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
logistic_model = LogisticRegression(max_iter=1000, random_state=42)

# Train the model
logistic_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = logistic_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2%}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 98.95%
Classification Report:
                precision    recall  f1-score   support

Critical Error       0.92      1.00      0.96        35
         Error       0.96      0.89      0.92        27
   HTTP Status       1.00      1.00      1.00       197
Resource Usage       1.00      1.00      1.00        35
Security Alert       1.00      0.99      0.99        87

      accuracy                           0.99       381
     macro avg       0.98      0.98      0.98       381
  weighted avg       0.99      0.99      0.99       381



In [21]:
import joblib
joblib.dump(logistic_model, '../models/log_classifier_model.pkl')

['../models/log_classifier_model.pkl']